In [1]:
%%capture
!pip install pip3-autoremove
!pip-autoremove torch torchvision torchaudio -y
!pip install torch torchvision torchaudio xformers --index-url https://download.pytorch.org/whl/cu121
!pip install unsloth

In [ ]:
from unsloth import FastLanguageModel
import torch
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "Nguyenhhh/Lamma_fi",     # Private model
    max_seq_length = 4096,

)

FastLanguageModel.for_inference(model)


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-06-18 06:50:43.573051: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750229443.763664      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750229443.819786      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


🦥 Unsloth Zoo will now patch everything to make training faster!


/usr/local/lib/python3.11/dist-packages/peft/config.py:162: UserWarning: Unexpected keyword arguments ['alpha_pattern', 'bias', 'corda_config', 'eva_config', 'exclude_modules', 'fan_in_fan_out', 'init_lora_weights', 'layer_replication', 'layers_pattern', 'layers_to_transform', 'loftq_config', 'lora_alpha', 'lora_bias', 'lora_dropout', 'megatron_config', 'megatron_core', 'modules_to_save', 'r', 'rank_pattern', 'target_modules', 'trainable_token_indices', 'use_dora', 'use_rslora'] for class PeftConfig, these are ignored. This probably means that you're loading a configuration file that was saved using a higher version of the library and additional parameters have been introduced since. It is highly recommended to upgrade the PEFT version before continuing (e.g. by running `pip install -U peft`).
  warnings.warn(


==((====))==  Unsloth 2025.6.2: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


...

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/peft/config.py:162: UserWarning: Unexpected keyword arguments ['corda_config', 'trainable_token_indices'] for class LoraConfig, these are ignored. This probably means that you're loading a configuration file that was saved using a higher version of the library and additional parameters have been introduced since. It is highly recommended to upgrade the PEFT version before continuing (e.g. by running `pip install -U peft`).
  warnings.warn(


adapter_model.safetensors:   0%|          | 0.00/336M [00:00<?, ?B/s]

Unsloth 2025.6.2 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096, padding_idx=128004)
        (layers): ModuleList(
          (0): LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=32, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=32, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear

In [3]:
import pandas as pd
import json

file_path = '/kaggle/input/privatetestlamma1000/chillprivatetest.json'
dff = pd.read_json(file_path, lines=True)

dff

,instruction,input
0,Based on the paragraph describing the image an...,CNN Should this NFL game be shown as a live br...
1,Based on the paragraph describing the image an...,"CNN On January 1, three women were killed in F..."
2,Based on the paragraph describing the image an...,Story highlightsIn Out device can be put on ne...
3,Based on the paragraph describing the image an...,CNN The 84th version of the NFL draft looks al...
4,Based on the paragraph describing the image an...,CNN US President Donald Trump announced on Fri...
...,...,...
1995,Based on the paragraph describing the image an...,Story highlightsRoger Federer s win over Andy ...
1996,Based on the paragraph describing the image an...,Story highlightsUS Open starts on August 29Gra...
1997,Based on the paragraph describing the image an...,Story highlightsAndy Murray begins his U.S. Op...
1998,Based on the paragraph describing the image an...,CNN It s the question many in tennis are askin...


In [4]:
df = dff[500:1000]

In [5]:
def generate_description(intructions, long_passage, model, tokenizer, device="cuda"):
    # Prompt theo định dạng Alpaca
    alpaca_prompt = """### Instruction:
{instruction}

### Input:
{input}

### Response:
{output}"""

    # Tạo prompt cụ thể
    prompt_text = alpaca_prompt.format(
        instruction=intructions,
        input=long_passage,
        output=""
    )

    # Tokenize và đẩy lên device
    inputs = tokenizer([prompt_text], return_tensors="pt").to(device)

    # Sinh đầu ra
    outputs = model.generate(
        input_ids=inputs.input_ids,
        attention_mask=inputs.attention_mask,
        max_new_tokens=1024,
        temperature=0.9,           # Tăng độ sáng tạo (mặc định là 1.0)
        top_p=0.9,                # Top-p sampling để đa dạng kết quả
        do_sample= True,           # Bắt buộc để dùng sampling thay vì greedy
        num_return_sequences=1,   # Chỉ tạo ra 1 kết quả duy nhất
        use_cache=True
    )

    # Giải mã kết quả
    result = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

    # Cắt chuỗi để lấy phần sau "### Response:"
    if "### Response:" in result:
        result = result.split("### Response:")[-1].strip()

    return result


In [6]:
from tqdm import tqdm

# Tạo một danh sách để chứa kết quả
results = []

# Duyệt từng dòng trong DataFrame với tqdm
for idx, row in tqdm(df.iterrows(), total=len(df), desc="Generating Responses"):
    instruction = row["instruction"]
    input_text = row["input"]

    # Gọi hàm generate_description
    response = generate_description(instruction, input_text, model, tokenizer, device="cuda")

    # Lưu kết quả
    results.append(response)

# Thêm cột kết quả mới vào DataFrame
df["response"] = results


Generating Responses: 100%|██████████| 500/500 [1:35:02<00:00, 11.40s/it]
/tmp/ipykernel_19/1542728279.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["response"] = results


In [7]:
df.to_csv('caption_Lamma.csv')